#### Note
This is not the version of the code which is used to make the csv files.

The results have been generated on data which has *not* been divided into slices and have *not* been interpolated.

The calculations are only done on the peak Holocene and LIG data

This notebook is for the confidence interval evaluation.

# Set up file 

In [1]:
import numpy as np
import pandas as pd
from netCDF4 import Dataset
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable  
import matplotlib
import scipy
import scipy.interpolate
import sys
sys.path.insert(0, '/srv/ccrc/data06/z5145948/Python/python_from_R/Holocene/sampled_models/plotting_files/')
from plott import plott
import scipy.interpolate
#from mpl_toolkits.basemap import Basemap
matplotlib.use('agg')
from Cross_section import Cross_section
from Proxy_graph_masked import Proxy_graph
import Config
#from Map_plot import Map_plot
from collections import Counter
import scipy.stats as stats
from pylab import *
rcParams['legend.numpoints'] = 1
from matplotlib.ticker import MaxNLocator
import time
from IPython import embed
import os
import ast
from matplotlib import gridspec

matplotlib.rcParams['text.usetex'] = True
matplotlib.rcParams['text.latex.preamble'] = [
    r'\usepackage{wasysym}',
    r'\usepackage{textcomp}']

import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 1200

%matplotlib inline

In [2]:
# Import standardised latitudes and depths
lat_standardised = np.genfromtxt('/srv/ccrc/data06/z5145948/Moving_water_mass/Data/latitude_levels.csv', delimiter=',')
dep_standardised = np.genfromtxt('/srv/ccrc/data06/z5145948/Moving_water_mass/Data/depth_levels.csv', delimiter=',')

# File saving location
overleaf_loc = '/home/z5145948/Dropbox/Apps/Overleaf/Paper_2/Figures'

# Set universal dividing depth
dividing_depth = 2500

# Set colors
alpha = 0.5
point_color_deep = (0,0,1,alpha)#'blue'
point_color_shallow = (1,0,0,alpha)#'red'
line_color_deep = 'cyan'
line_color_shallow = 'magenta'

# Font size
fontsize = 20

font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : fontsize}
matplotlib.rc('font', **font)

# Separate into different areas
point_size_power = 1
points_size_multiplier = 10
south_lat_max = -15
north_lat_min = 15
lw = 4
edgewidth=0.5
size = 40
weighting = False

slice_width = 1.0



# Define Wrangling Functions

In [3]:
####3 reading data function

def read_data(folder):

    # Import simulation details (summary) as dataframe
    summary = pd.read_csv(folder + '_summary.txt', delimiter = ' ')

    # Import simulation outputs
    fh = Dataset(folder + 'output.nc')
    proxy_simulations = fh.variables['var1_1'][:] 

    # Import samples (proxy data)
    samples = pd.read_csv( folder + '_samples.txt', delimiter = ' ')

    # Drop all unnecessary rows in summary
    summary = summary.drop(['type', 'Row.names', 'reps', 'model', 'dataset', 'filename', 'success.rate'], axis = 1)

    # Convert run.no to netcdf file index
    summary['run.no'] = summary['run.no'] - 1
    samples['run.no'] = samples['run.no'] - 1
    
    return(proxy_simulations, samples, summary)

In [4]:
# Define a function for reading and including cores that are in the Oliver compilation but not in the Peterson data set

def Oliver_cores(minn, maxx):
    import pandas as pd
    import numpy as np
    folder_location = '/srv/ccrc/data06/z5145948/Moving_water_mass/Data/Core_files/'
    file_mat = ['GeoB4403_2.txt',
             'GeoB1028_5.txt',
             'GeoB2109_1.txt',
             'GeoB3801_6.txt',
             'V22_38.txt',
             'V28_56.txt',
             'V27_20.txt',
             'RC12_339.txt',
             'V32_128.txt',
             'GIK16772_1.txt',
             'MD96_2080.txt',
             'MD06_3018.txt',
             'NEAP18K.txt',
             'KNR140_37JPC.txt']


    location = ['Atlantic,','Atlantic,','Atlantic,','Atlantic,','Atlantic,','Atlantic,','Atlantic,','Indian,','Pacific,','Atlantic,','Atlantic,','Pacific,','Atlantic,','Atlantic,']

    oliver_data = []

    i = 0

    while i < len(file_mat):
        with open(folder_location + file_mat[i]) as f:
            for line in f:
                oliver_data.append(location[i] + line)
        i += 1

    df = pd.DataFrame([sub.split(",") for sub in oliver_data])

    df.columns = ['Location','Core','Lat','Lon','Dep','Core depth','age','Species','pl1','pl2','d18O benthic','d13C']

    df = df[['Core','Location','Lat','Lon','Dep','d13C','age']]

    df['d13C'] = [i.rstrip() for i in df['d13C']]

    df = df[df['age'].astype(float) > float(minn)]
    df = df[df['age'].astype(float) < float(maxx)]

    ################################# other data                                                                                                                                                                                                                                  

    file_mat = ['CH69_K09.txt',
    'MD03_2664.txt',
    'MD95_2042.txt',
    'U1308.txt',
    'ODP1063.txt']

    locations = ['CH69_K09\tAtlantic\t41.75\t-47.35\t4100\t',
              'MD03_2664\tAtlantic\t57.439000\t-48.605800\t3442.0\t',
              'MD95_2042\tAtlantic\t37.799833\t-10.166500\t3146.0\t',
              'U1308\tAtlantic\t49.877760\t-24.238110\t3871.0\t',
              'ODP1063\tAtlantic\t33.683333\t-57.616667\t4584\t']

    other_data = []
    i = 0

    while i < len(file_mat):
        with open(folder_location + file_mat[i]) as f:
            for line in f:
                other_data.append(locations[i]+line)
        i += 1

    df2 = pd.DataFrame([sub.split("\t") for sub in other_data])

    df2.columns = ['Core','Location','Lat','Lon','Dep','Core Depth','age','d13C']

    df2 = df2[['Core','Location','Lat','Lon','Dep','d13C','age']]

    df2['d13C'] = [i.rstrip() for i in df2['d13C']]

    df2 = df2[df2['age'].astype(float) > float(minn)]
    df2 = df2[df2['age'].astype(float) < float(maxx)]

    results = pd.concat([df, df2])

    results = results[results['d13C'] != '']

    results['d13C'] = results['d13C'].astype(float)
    results['Lat'] = results['Lat'].astype(float)
    results['Lon'] = results['Lon'].astype(float)
    results['Dep'] = results['Dep'].astype(float)

    return(results)    
    

In [5]:

def pl_cores(minn, maxx):

    names = ['Core', 'Location', 'Lat', 'Lon', 'Dep']

    # Read in the data
    indopac = pd.read_table("../Moving_water_mass/Data/Core_files/indopac_core_data_LS16.txt", delimiter = ',', names = names)
    atl = pd.read_table("../Moving_water_mass/Data/Core_files/atl_core_data_LS16.txt", delimiter = ',', names = names)
    add = pd.read_table("../Moving_water_mass/Data/Core_files/Additional_core_locations.txt", delimiter = ',', usecols = [0, 1, 2, 3, 4], names = names)

    # Join all into a single dataframe
    df = indopac.append(atl)
    df = df.append(add)
    df = df.reset_index(drop = True)

    df['d18O names'] = df['Core'] + '_ageLS16.txt'
    df['d13C names'] = df['Core'] + '_d13C.txt'

    # Loop over the dataset and interpolate each core
    i = 0
    results_dict = {}

    while i < df.count()[0]:

        try:
            df_d18O = pd.read_table('../Moving_water_mass/Data/Core_files/' + df.loc[i]['d18O names'], delim_whitespace = True, names = ['depth', 'age'], skip_blank_lines = True, na_values = 'NAN')
        except:
            i += 1
            continue
        try:
            df_d13C = pd.read_table('../Moving_water_mass/Data/Core_files/' + df.loc[i]['d13C names'], delim_whitespace = True, names = ['depth', 'd13C'], skip_blank_lines = True, na_values = 'NAN')
        except:
            i += 1
            continue

        df_d18O = df_d18O.dropna(subset = ['age']) 
        df_d13C = df_d13C.dropna(subset = ['d13C'])

        df_d18O = df_d18O.reset_index(drop = True)
        df_d13C = df_d13C.reset_index(drop = True)

        interp = scipy.interpolate.interp1d(df_d18O['depth'], df_d18O['age'], bounds_error = True)
        try:
            df_d13C['age'] = interp(df_d13C['depth'])
        except:
            try:
                interp2 = scipy.interpolate.interp1d(df_d18O['depth'], df_d18O['age'], bounds_error = False)
                df_d13C['age'] = interp2(df_d13C['depth'])
            except:
                i += 1
                continue

        df_d13C = df_d13C.dropna(subset = ['age'])
        df_d13C = df_d13C.reset_index(drop = True)

        df_d13C = df_d13C[df_d13C['age'] > minn]
        df_d13C = df_d13C[df_d13C['age'] < maxx]

        if len(df_d13C) > 0:
            df_results = df.drop(['d18O names', 'd13C names'], axis = 1)
            df_results = df_results.loc[df_results.index.repeat(len(df_d13C))].loc[[i]]

            df_d13C = df_d13C.drop(['depth'], axis = 1)

            df_results = df_results.reset_index(drop = True).join(df_d13C.reset_index(drop = True))
            results_dict.update({
                df_results.Core[0] : df_results.drop(['Core'], axis = 1)
            })

        i += 1
        
    df_results = pd.concat(results_dict).reset_index()
    df_results = df_results.rename(columns = {'level_0' : 'Core'})
    df_results = df_results.drop(['level_1'], axis = 1)

    return(df_results)
    

In [6]:

def slicing_data(time_min,time_max):
    # ################## read in the oliver data using the predefined function

    # Age range to look over

    df_pl = pl_cores(time_min,time_max)

    df_oliver = Oliver_cores(time_min,time_max)

    ###################3 join the dataframes

    df_results = df_pl.append(df_oliver)

    #################### get only the atlantic cores

    df = df_results.reset_index(drop = True)
    df['Dep'] = abs(df['Dep'])

    df_atl = df[df['Location'] == 'Atlantic']
    df_atl = df_atl.reset_index(drop = True)
    df_atl['age'] = df_atl.age.astype(float)
        
    return(df_atl)

In [7]:

def Atlantic_regions(proxy_compilation):
    # Get volume weight average with time slices
    interpolated_samples['weights'] = np.nan
    interpolated_samples['regions'] = np.nan    

    # NEA
    interpolated_samples.loc[((interpolated_samples['Lon'] < 20) | (interpolated_samples['Lon'] > (-33))) & (interpolated_samples['Lat'] > 0.1),
                            'weights'] = 4.3
    interpolated_samples.loc[((interpolated_samples['Lon'] < 20) | (interpolated_samples['Lon'] > (-33))) & (interpolated_samples['Lat'] > 0.1),
                            'regions'] = 'NEA'
    

    # NWA
    interpolated_samples.loc[(interpolated_samples['Lon'] < (-33)) & (interpolated_samples['Lon'] > (-180)) & (interpolated_samples['Lat'] > 0.1),
                             'weights'] = 4.9
    interpolated_samples.loc[(interpolated_samples['Lon'] < (-33)) & (interpolated_samples['Lon'] > (-180)) & (interpolated_samples['Lat'] > 0.1),
                             'regions'] = 'NWA'

    # SEA
    interpolated_samples.loc[((interpolated_samples['Lon'] < 30) | (interpolated_samples['Lon'] > (-14.6))) & (interpolated_samples['Lat'] < 0) & (interpolated_samples['Lat'] > -55),
                             'weights'] = 3.5
    interpolated_samples.loc[((interpolated_samples['Lon'] < 30) | (interpolated_samples['Lon'] > (-14.6))) & (interpolated_samples['Lat'] < 0) & (interpolated_samples['Lat'] > -55),
                             'regions'] = 'SEA'
    
    # SA
    interpolated_samples.loc[((interpolated_samples['Lon'] < 30) | (interpolated_samples['Lon'] > (-22))) & (interpolated_samples['Lat'] < -40) & (interpolated_samples['Lat'] > -55),
                            'weights'] = 0.7
    interpolated_samples.loc[((interpolated_samples['Lon'] < 30) | (interpolated_samples['Lon'] > (-22))) & (interpolated_samples['Lat'] < -40) & (interpolated_samples['Lat'] > -55),
                            'regions'] = 'SA'    

    # SWA
    interpolated_samples.loc[(interpolated_samples['Lon'] > (-60)) & (interpolated_samples['Lon'] < (-14.6)) & (interpolated_samples['Lat'] < 0) & (interpolated_samples['Lat'] > -55),
                            'weights'] = 5.0
    interpolated_samples.loc[(interpolated_samples['Lon'] > (-60)) & (interpolated_samples['Lon'] < (-14.6)) & (interpolated_samples['Lat'] < 0) & (interpolated_samples['Lat'] > -55),
                            'regions'] = 'SWA'    
    
    return(interpolated_samples)
    
def Pacific_regions(proxy_compilation):
    # Get volume weight average with time slices
    interpolated_samples['weights'] = np.nan
    interpolated_samples['regions'] = np.nan      

    # North
    interpolated_samples.loc[(interpolated_samples['Lat'] > 0) ,'weights'] = 21.2
    interpolated_samples.loc[(interpolated_samples['Lat'] > 0) ,'regions'] = 'NP'    

    # South
    interpolated_samples.loc[(interpolated_samples['Lat'] < 0) ,'weights'] = 23.9
    interpolated_samples.loc[(interpolated_samples['Lat'] < 0) ,'regions'] = 'SP'
    
    return(interpolated_samples)

def Indian_regions(proxy_compilation):
    # Get volume weight average with time slices # Use a single region for the Indian Ocean
    interpolated_samples['weights'] = 1
    interpolated_samples['regions'] = 'I'
    
    return(interpolated_samples)


In [8]:
def averaging_by_region(proxy_compilation):
    # group the cores based on the age and the region (indicated by weights)
    grouped_by_age_region = proxy_compilation.groupby(['regions'])
    
    averaged_by_age_region = {}
    stdev_by_age_region = {}
    measurement_count = {}
    core_count = {}
    CI_by_age_region = {}
    
    for key, group in grouped_by_age_region:
        
        # find group means
        averaged_by_age_region.update({
            key: np.mean(group)
        })
        # find total number of cores
        core_count.update({
            key: len(group['count'].dropna())
        }) 
        # find total measurement count
        measurement_count.update({
            key: np.nansum(group['count'])
        })
        # find the standard deviation across the slices and cores
        stdev_by_age_region.update({
            key: np.std(group['d13C'])
        })
        # express variation as a confidence interval
        CI_by_age_region.update({
            key: 1.96 * np.std(group['d13C'])/(np.sqrt(len(group['count'].dropna()))) #CI of 95%, 1.96 is zscore
        })        
        
    # Convert dictionaries to dataframes
    averaged_by_age_region = pd.concat(averaged_by_age_region,axis=1).T.reset_index(drop=False).rename({'level_1':'regions'},axis=1).drop('level_0',axis=1)
    measurement_count = pd.DataFrame.from_dict(measurement_count,orient='index')
    core_count = pd.DataFrame.from_dict(core_count,orient='index')
    stdev_by_age_region = pd.DataFrame.from_dict(stdev_by_age_region,orient='index')
    CI_by_age_region = pd.DataFrame.from_dict(CI_by_age_region,orient='index')    
    
    # Add columns of number of cores and number of measurements to the dataframe
    averaged_by_age_region['measurement_count'] = list(measurement_count[0])
    averaged_by_age_region['core_count'] = list(core_count[0])  
    averaged_by_age_region['slice_stdev'] = list(stdev_by_age_region[0])
    averaged_by_age_region['CI'] = list(CI_by_age_region[0])
        
    return(averaged_by_age_region)


In [15]:

def sort_data(time_min,time_max,region_func,interp_data_name,notinterp_data_name):

    # Slice the data
    proxy_compilation = slicing_data(time_min,time_max)

    # get region weights
    proxy_compilation = region_func(proxy_compilation)
    
    # Get regional average and ocean basin averages
    averaged_by_age_region = averaging_by_region(proxy_compilation)

    # drop weights (because I'm not using them now)
    interpolated_samples = interpolated_samples.drop('weights',axis=1)
    
    return(averaged_by_age, averaged_by_age_region)


# Run the functions

In [16]:
# peak LIG and peak Holocene time periods
LIG_min = 120
LIG_max = 125
Hol_min = 4
Hol_max = 7

# set up other variables

# Holocene, Atlantic
Hol_atl_nointerp_data_name = "Data/Atlantic_Holocene_profile_PL.csv"
Hol_atl_interp_data_name = "/srv/ccrc/data06/z5145948/Moving_water_mass/Data/interpolated_Hol_Atlantic.csv"

# LIG, Atlantic
LIG_atl_nointerp_data_name = "Data/Atlantic_LIG_profile_PL.csv"
LIG_atl_interp_data_name = "/srv/ccrc/data06/z5145948/Moving_water_mass/Data/interpolated_LIG_Atlantic.csv"

# Holocene, Pacific
Hol_pac_nointerp_data_name = "Data/Pacific_Holocene_profile_PL.csv"
Hol_pac_interp_data_name = "/srv/ccrc/data06/z5145948/Moving_water_mass/Data/interpolated_Hol_Pacific.csv"

# LIG, Pacific
LIG_pac_nointerp_data_name = "Data/Pacific_LIG_profile_PL.csv"
LIG_pac_interp_data_name = "/srv/ccrc/data06/z5145948/Moving_water_mass/Data/interpolated_LIG_Pacific.csv"

# Holocene, Indian
Hol_ind_nointerp_data_name = "Data/Indian_Holocene_profile_PL.csv"
Hol_ind_interp_data_name = "/srv/ccrc/data06/z5145948/Moving_water_mass/Data/interpolated_Hol_Indian.csv"

# LIG, Indian
LIG_ind_nointerp_data_name = "Data/Indian_LIG_profile_PL.csv"
LIG_ind_interp_data_name = "/srv/ccrc/data06/z5145948/Moving_water_mass/Data/interpolated_LIG_Indian.csv"

In [17]:
# Run function for each scenario (three oceans, two time periods)

interpolated_results_dict = {}
average_age_interpolated_results_dict = {}
average_age_region_interpolated_results_dict = {}

for label,time_min,time_max,region_func,interp_data_name,notinterp_data_name in zip(
    ['Hol_atl','LIG_atl','Hol_pac','LIG_pac','Hol_ind','LIG_ind'],
    [Hol_min,LIG_min,Hol_min,LIG_min,Hol_min,LIG_min],
    [Hol_max,LIG_max,Hol_max,LIG_max,Hol_max,LIG_max],
    [Atlantic_regions,Atlantic_regions,Pacific_regions,Pacific_regions,Indian_regions,Indian_regions],
    [Hol_atl_interp_data_name,LIG_atl_interp_data_name,Hol_pac_interp_data_name,LIG_pac_interp_data_name,Hol_ind_interp_data_name,LIG_ind_interp_data_name],
    [Hol_atl_nointerp_data_name,LIG_atl_nointerp_data_name,Hol_pac_nointerp_data_name,LIG_pac_nointerp_data_name,Hol_ind_nointerp_data_name,LIG_ind_nointerp_data_name]):
    
    # Run function
    averaged_by_age, averaged_by_age_region, interpolated_samples = sort_data(
        time_min,time_max,region_func,interp_data_name,notinterp_data_name
    )
    
    # Add results to dictionary
    interpolated_results_dict.update({
        label : interpolated_samples
    })
    average_age_interpolated_results_dict.update({
        label : averaged_by_age
    })   
    average_age_region_interpolated_results_dict.update({
        label : averaged_by_age_region
    })     

NameError: name 'interpolated_samples' is not defined

In [ ]:
# Create results dataframes from dictionaries
df_interpolated_results = pd.concat(interpolated_results_dict).reset_index(drop=True)
df_average_age_interpolated_results = pd.concat(average_age_interpolated_results_dict).reset_index().drop('level_1',axis=1).rename(columns=
                                                                                 {'level_0':'location'})
df_average_age_region_interpolated_results = pd.concat(average_age_region_interpolated_results_dict).reset_index().drop('level_1',axis=1).rename(columns=
                                                                                 {'level_0':'location'})

In [ ]:
# find average results of peak period
df_LIG_peak = df_average_age_region_interpolated_results[(df_average_age_region_interpolated_results['age'] > LIG_peak_min) & (df_average_age_region_interpolated_results['age'] < LIG_peak_max)]
df_Hol_peak = df_average_age_region_interpolated_results[(df_average_age_region_interpolated_results['age'] > Hol_peak_min) & (df_average_age_region_interpolated_results['age'] < Hol_peak_max)]


In [ ]:
# Average over all time slices for LIG 
LIG_peak_averaged_dict = {}
for index,group in df_LIG_peak.groupby(['location','regions']):
    LIG_peak_averaged_dict.update({
        index:pd.DataFrame(np.mean(group)).T
    })
df_LIG_peak_averaged = pd.concat(LIG_peak_averaged_dict)
    
# Average over all time slices for Holocene
Hol_peak_averaged_dict = {}
for index,group in df_Hol_peak.groupby(['location','regions']):
    Hol_peak_averaged_dict.update({
        index:pd.DataFrame(np.mean(group)).T
    })    
df_Hol_peak_averaged = pd.concat(Hol_peak_averaged_dict)    
    
# Format dataframes for merge
df_Hol_peak_averaged = df_Hol_peak_averaged.reset_index().drop(['weights','age','count','level_2','level_0'],axis=1)
df_Hol_peak_averaged = df_Hol_peak_averaged.rename(columns={'level_1':'Location'})
df_Hol_peak_averaged.set_index(['Location'],inplace=True)

df_LIG_peak_averaged = df_LIG_peak_averaged.reset_index().drop(['age','count','level_2','level_0'],axis=1)
df_LIG_peak_averaged = df_LIG_peak_averaged.rename(columns={'level_1':'Location'})
df_LIG_peak_averaged.set_index(['Location'],inplace=True)

In [ ]:
test = df_Hol_peak_averaged.join(df_LIG_peak_averaged,lsuffix=' Holocene',rsuffix=' LIG')

In [ ]:
test['d13C LIG'] - test['d13C Holocene']

In [ ]:
fig = plt.figure(figsize=(10,10))

ax1 = subplot2grid((1,1),(0,0))
ax1 = subplot2grid((1,1),(0,0))

plt.scatter(test.index,test['d13C LIG'])
plt.scatter(test.index,test['d13C Holocene'])

plt.errorbar(test.index,test['d13C LIG'],yerr=test['CI LIG'],ls='none')
plt.errorbar(test.index,test['d13C Holocene'],yerr=test['CI Holocene'],ls='none')

plt.bar()

plt.show()

In [ ]:
test